In [1]:
print("hi")

hi


In [2]:
#Load edges and nodes

import numpy as np
from load_data import *

nodes_array, edge_array = assert_edges_are_within_first_cluster()

node_lookup_dict = {}

min_index = np.amin(nodes_array)
max_index = np.amax(nodes_array)


for node in nodes_array:
    node_lookup_dict[node] = 1

98753it [00:00, 925833.02it/s]
2004it [00:00, 1097738.70it/s]


In [3]:
#Get dois from the exosome csv

from load_data import *

import csv

csv_file_path = 'exosome.csv'

# Open the CSV file for reading
with open(csv_file_path, 'r') as csv_file:
    csv_reader = csv.reader(csv_file)
    dois = {}
    for line_number, row in tqdm(enumerate(csv_reader), total = 14695476):  # 'total' is the total number of iterations
        
        if line_number <min_index or line_number > max_index:
            continue
            
        id = int(row[0])
        doi = row[2]
        
        if id in node_lookup_dict:
            node_lookup_dict[id] = doi



 82%|████████▏ | 12059410/14695476 [00:26<00:05, 451026.13it/s]


KeyboardInterrupt: 

In [ ]:
import json

with open('first_cluster_dois.json', 'w') as json_file:
    json.dump(node_lookup_dict, json_file, indent=4)
    


In [ ]:
import requests
import xmltodict

doi_dict = {}

#Get dois
def fetch_pmid_from_doi(doi='10.1073/pnas.0510928103'):

    pmid_dict = {}
    request_str = 'https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/?tool=my_tool&email=my_email@example.com&ids='
    request_str += str(doi)
    
    response = requests.get(request_str)
    response_text = response.text
    
    if response.status_code == 200:
        index = response_text.index('pmid=')


        pmid_whole = response_text[index:].split(' ')[0]
        pmid = int(pmid_whole[:-1].split('\"')[1])

        return pmid
    
    else:
        return 0
    

for key in tqdm(node_lookup_dict.keys()):
    doi_dict[key] = fetch_pmid_from_doi(key)
    

In [ ]:
#All Pmids are node numbers!

for key, value in doi_dict.items():
    assert key == value
    

with open('first_cluster_pmid.json', 'w') as json_file:
    json.dump(doi_dict, json_file, indent=4)
    

In [ ]:
import json

with open('first_cluster_pmid.json', 'r') as json_file:
    doi_dict = json.load(json_file)
    
pmid_dict = doi_dict

In [ ]:
#Save all xmls
all_xmls = {}
def save_all_xmls(pmid):
    wait = 0.25
    time.sleep(wait)
    pmid_dict = {}
    request_str  = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id='
    request_str += str(pmid)
    
    response = requests.get(request_str)
    response_text = response.text
    
    try:
        xml_dict = xmltodict.parse(response_text)
        
    except:
        time.sleep(2 * wait)
        return fetch_metadata_from_pmid(pmid)
    
    return response_text

for id in tqdm(pmid_dict):
    all_xmls[id] = save_all_xmls(id)

with open('first_cluster_xmls.json', 'w') as json_file:
    json.dump(all_xmls, json_file, indent=4)
    




In [ ]:
#Load all XMLs
import json

with open('first_cluster_xmls.json', 'r') as json_file:
    all_xmls = json.load(json_file)

In [ ]:
#Mispelled
import time
import requests
import xmltodict

parsed_error = []
failing_grant = []
dict_of_interest = {'keyword': 0, "grant": 0, "abstract": 0, "mesh": 0, "chemicallist":0, 'datecompleted': 0}


def fetch_metadata_from_pmid(response_text, idx, pmid): 
    global dict_of_interest
    mesh_headings = []
    grants = []
    year = ""
    journal_ISSN = ""
    abstract = ""
    chemical_list = []
    meta_data = {}
    
    try:
        xml_dict = xmltodict.parse(response_text)
        
        for key_of_interest in dict_of_interest.keys():
            if key_of_interest in str(xml_dict).lower():

                dict_of_interest[key_of_interest] += 1

        #Date completed or revised?
        #print(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation'].keys())
        
        
        try:
            if 'DateCompleted' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']:
                if 'Year' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['DateCompleted']:
                    year = xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['DateCompleted']['Year']

            else:
    #             print(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation'])
                  pass
    
        except:
            pass


        
        try:
            
            if 'ISSN' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['Journal'].keys():
                journal_ISSN = xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['Journal']['ISSN']['#text']
        
        except:
            pass
        #print(len(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['GrantList']['Grant']))
        
        
        #Grant
        #Very few grants don't have grant id's
        #Grant institute could also be relevant
        #TODO: collect missing grant id

        try:
            if 'GrantList' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']:
                for grant in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['GrantList']['Grant']:
                    if 'GrantID' in grant and 'Agency' in grant:
                        grants.append((grant['GrantID'],grant['Agency']))
                        
        except:
            pass
        
        
        try:
            
            if 'Abstract' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article'].keys():
                if 'AbstractText'  in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['Abstract'].keys():
                    abstract = xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['Abstract']['AbstractText']
        except:
            pass
        
 

        #print(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['Article']['GrantList']['Grant'][0])
        
        #MeSH heading  
        
        try:
        #print(len(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['MeshHeadingList']['MeshHeading']))
        
            #DescriptorName? Is a term
            if 'MeshHeadingList' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']:
                for mesh in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['MeshHeadingList']['MeshHeading']:
    #                 print(mesh['DescriptorName'])
    #                 print(mesh['DescriptorName']['@UI'])
    #                 print(mesh['DescriptorName']['#text'])

                    mesh_headings.append((mesh['DescriptorName']['@UI'], mesh['DescriptorName']['#text']))
        except:
            pass
        
        
        try:
        #print(len(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['MeshHeadingList']['MeshHeading']))
        
            #DescriptorName? Is a term
            if 'ChemicalList' in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']:
#                print(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']
# #                      ['ChemicalList']['Chemical']['NameOfSubstance'])
               print(len(xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']
                     ['ChemicalList']['Chemical']))
    
               for substance in xml_dict['PubmedArticleSet']['PubmedArticle']['MedlineCitation']['ChemicalList']['Chemical']:
                    print("wtf")
               
               print(chemical_list)
                
               
              

        except:
            pass
        


        #References and history
        #print(xml_dict['PubmedArticleSet']['PubmedArticle']['PubmedData'].keys())


    except:
        parsed_error.append(idx)
        pass


 
        
    meta_data = {'mesh': mesh_headings, 'grants': grants, 'year': year, 'journal': journal_ISSN, 'abstract': abstract }
    return meta_data

metadata_dict = {}

# [166, 719, 1672, 1918] odd

idx = 0
for key, value in tqdm(all_xmls.items()):
    metadata_dict[key] = fetch_metadata_from_pmid(value, idx, key)
    idx += 1
    


print(f'failing grants: {len(failing_grant)}')
print(f'failed parse {len(parsed_error)}')
print(f'Occured keyword dict: {dict_of_interest}')

In [ ]:
#### Count incomplete data

features = {'mesh': 0, 'grants': 0, 'year': 0, 'journal': 0, 'abstract': 0}
#Mesh terms are sometimes unrelated

mesh_length = []

for feature in features.keys():
    for _, meta in metadata_dict.items():
        if len(meta[feature]) > 0:
            features[feature] += 1
        elif feature == 'mesh':
            mesh_length.append(len(meta[feature]))
            
print("Ocurred features ", features)
print("avg mesh length ", np.mean(mesh_length))


In [ ]:
import json

with open('first_cluster_metadata.json', 'w') as json_file:
    json.dump(metadata_dict, json_file, indent=4)
    


In [ ]:
def report_stats(vals):
    median = np.median(vals)
    mean = np.mean(vals)
    min = np.amin(vals)
    max = np.amax(vals)
    total = np.sum(vals)

    print(f'median {median} mean {mean} min {min} max {max}, total {total}')

    

def calculate_distance(feature):
    distance_matrix = np.zeros((len(nodes_array), len(nodes_array))) 
    all_values = []
    
    if feature == 'year':
        for i in tqdm(range(distance_matrix.shape[0])):
            for j in range(i):
                pmid1 = str(nodes_array[i])
                pmid2 = str(nodes_array[j])
                
                year1 = metadata_dict[pmid1][feature]
                year2 = metadata_dict[pmid2][feature]
                
                if len(year1) > 0 and len(year2)>0:
                    distance_matrix[i][j]= np.abs(int(year1)-int(year2))
                    all_values.append(distance_matrix[i][j])
                
                else:
                    distance_matrix[i][j]= -1
        
        distance_matrix = distance_matrix + distance_matrix.transpose()
        return distance_matrix, all_values

distance_matrix, all_values = calculate_distance('year')
report_stats(all_values)

In [ ]:
print(metadata_dict)